In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import pickle
from scipy.stats import norm

In [22]:
# Data required from previous parts

# 1. Discount Curve (OIS & LIBOR)
with open('data/ois_discount_curve.pkl', 'rb') as f:
    ois_discount_curve = pickle.load(f)
with open('data/libor_discount_curve.pkl', 'rb') as f:
    libor_discount_curve = pickle.load(f)

# 2. Calibrated SABR params
__tenors__ = [1,2,3,5,10]
__expiries__ = [1,5,10]
sabr_alpha_df = pd.read_excel("data/sabr_calibrated.xlsx", sheet_name="sabr_alpha", index_col=0)
sabr_alpha_df.index = __expiries__
sabr_alpha_df.columns = __tenors__
sabr_rho_df = pd.read_excel("data/sabr_calibrated.xlsx", sheet_name="sabr_rho", index_col=0)
sabr_rho_df.index = __expiries__
sabr_rho_df.columns = __tenors__
sabr_nu_df = pd.read_excel("data/sabr_calibrated.xlsx", sheet_name="sabr_nu", index_col=0)
sabr_nu_df.index = __expiries__
sabr_nu_df.columns = __tenors__

# 3. CMS rates
cms_rates_df = pd.read_csv("data/cms_rate.csv", index_col=0)
cms_rates_df.columns = cms_rates_df.columns.astype('int64')


In [31]:
# Option pricers

def Black76Call(F, K, sigma, T, D=1.0):
    d1 = (np.log(F/K)+0.5*sigma*sigma*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return D*(F*norm.cdf(d1)-K*norm.cdf(d2))

def Black76Put(F, K, sigma, T, D=1.0):
    return Black76Call(F, K, sigma, T, D) - D*(F-K)

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom
    return sabrsigma

The PDF of swap rate can be derived from the 2nd order derivative of IRR-settled swaption price
\begin{equation*}
\begin{split}
    f(K) &= \frac{\partial^2 V^{IRR}}{\partial K^2}\dfrac{1}{D(0,T)\cdot IRR(K)}
\end{split}
\end{equation*}
Using static replication approach and let $h(K)=\dfrac{g(K)}{IRR(K)}$, we have
\begin{equation*}
\begin{split}
    V_0 &= D(0,T) \int_0^\infty g(K)f(K)dK \\
    &= \int_0^F h(K)\frac{\partial^2 V^{rec}_0}{\partial K^2}dK+\int_F^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK
\end{split}
\end{equation*}
For option (1) that pays $g(K) = K^{1/p}-0.04^{1/q}$
\begin{equation*}
\begin{split}
    V_0 &= D(0,T)g(F) + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
\end{split}
\end{equation*}
For option (2) that pays $g(K) = (K^{1/p}-0.04^{1/q})^+$, when $K <= x* = 0.04^{p/q},g(K) = 0$  

When $x* <= F$
\begin{equation*}
\begin{split}
    V_0 &= \int_{x*}^F h(K)\frac{\partial^2 V^{rec}_0}{\partial K^2}dK+\int_F^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK \\
    &= D(0,T)g(F) + \int_{x^*}^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK \\
    &\;\;\;\;\;\;\;\;\;\;+ h'(x*)V^{rec}(x*)
\end{split}
\end{equation*}

When $x* > F$
\begin{equation*}
\begin{split}
    V_0 &= \int_{x*}^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK \\
    &= h'(x*)V^{pay}(x*) + \int_{x*}^\infty h''(K) V^{pay}(K) dK
\end{split}
\end{equation*}

Here,
\begin{equation*}
\begin{split}
    g(K) &= K^{1/p}-0.04^{1/q} \\
    g'(K) &= \dfrac{1}{p}K^{1/p-1} \\
    g''(K) &= \dfrac{(1-p)}{p^2}K^{1/p-2}
\end{split}
\end{equation*}

In [53]:
def IRR_0(K, m, N):
    return 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )

def IRR_1(K, m, N):
    return -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)

def IRR_2(K, m, N):
    return -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)

def g_0(K, p, q):
    return K**(1.0/p)-0.04**(1.0/q)
    # return K**0.25-0.04**0.5 # test

def g_1(K, p, q):
    return 1.0/p*K**(1.0/p-1)
    # return 0.25*K**(-0.75) # test

def g_2(K, p, q):
    return 1.0/p*(1.0/p-1)*K**(1.0/p-2)
    # return (-0.75)*0.25*K**(-1.75) # test

def h_0(K, m, N, p, q):
    return g_0(K, p, q) / IRR_0(K, m, N)

def h_1(K, m, N, p, q):
    return (IRR_0(K, m, N)*g_1(K, p, q) - g_0(K, p, q)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2

def h_2(K, m, N, p, q):
    return ((IRR_0(K, m, N)*g_2(K, p, q) - IRR_2(K, m, N)*g_0(K, p, q) - 2.0*IRR_1(K, m, N)*g_1(K, p, q))/IRR_0(K, m, N)**2 \
            + 2.0*IRR_1(K, m, N)**2*g_0(K, p, q)/IRR_0(K, m, N)**3)

def option_pricer_1(F, iv, Tn, N, freq, p, q):
    D_T = libor_discount_curve(Tn)
    F_max = F + (4 * iv(F) * np.sqrt(Tn))/100
    # F_max = F+0.02
    I_rec = quad(lambda x: h_2(x, freq, N, p, q) * Black76Put(F, 
                                                              x, 
                                                              iv(x), 
                                                              Tn),
                 1E-10, F)[0]
    I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                               x, 
                                                               iv(x), 
                                                               Tn),
                 F, F_max)[0]
    return D_T*g_0(F, p, q) + IRR_0(F, freq, N)*(I_rec + I_pay)

def option_pricer_2(F, iv, Tn, N, freq, p, q):
    D_T = libor_discount_curve(Tn)
    F_max = F + (4 * iv(F) * np.sqrt(Tn))/100
    # F_max = F+0.02
    x_star = 0.04**(p/q) # g(K) > 0 when K > x_star
    if x_star <= F:
        I_rec = quad(lambda x: h_2(x, freq, N, p, q) * Black76Put(F, 
                                                                  x, 
                                                                  iv(x),
                                                                  Tn),
                    x_star, F)[0]
        I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                                   x, 
                                                                   iv(x),
                                                                   Tn),
                    F, F_max)[0]
        extra_term = h_1(x_star, freq, N, p, q) * Black76Put(F, 
                                                             x_star, 
                                                             iv(x_star),
                                                             Tn)
        return D_T*g_0(F, p, q) + IRR_0(F, freq, N)*(I_rec + I_pay + extra_term)
    else:
        I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                                   x, 
                                                                   iv(x),
                                                                   Tn),
                    x_star, F_max)[0]
        extra_term = h_1(x_star, freq, N, p, q) * Black76Call(F, 
                                                              x_star, 
                                                              iv(x_star),
                                                              Tn)
        return IRR_0(F, freq, N)*(I_pay + extra_term)

In [54]:
# input
Tn = 5
N = 10
F = cms_rates_df[N][Tn]
alpha = sabr_alpha_df[N][Tn]
beta = 0.9
rho = sabr_rho_df[N][Tn]
nu = sabr_nu_df[N][Tn]
iv = lambda x: SABR(F, x, Tn, alpha, beta, rho, nu)

# output
pv1 = option_pricer_1(F, iv, 
                Tn=5, N=10, freq=2, p=4, q=2)
pv2 = option_pricer_2(F, iv, 
                Tn=5, N=10, freq=2, p=4, q=2)

print(f"""
==Input==
Forward Swap Rate = {F}
SABR model parameters:
  alpha = {alpha}
  beta = {beta}
  rho = {rho}
  nu= {nu}
  
==Output==
Contract 1 PV = {pv1}
Contract 2 PV = {pv2}
""")


==Input==
Forward Swap Rate = 0.1068986395072102
SABR model parameters:
  alpha = 0.171354499666421
  beta = 0.9
  rho = -0.3971619656711662
  nu= 0.5433373638486673
  
==Output==
Contract 1 PV = 0.29956560522033654
Contract 2 PV = 0.3134831550006551



<!-- When payoff is
$$
    g(S) = (S^{1/p} - 0.04^{1/q})^+\;\;\;\;\text{ where } p=4,\;q=2
$$
Let $S^{1/p}=X, 0.04^{1/q}=K$, the payoff and option pricer turn into
\begin{equation*}
\begin{split}
    g(K, X) &= (X - K)^+ \\
    V_0 &= D(0,T) \int_0^\infty g(s)\cdot f(s)ds \\
    &= D(0,T) \int_K^\infty g(K, x)\cdot f(x^p)\cdot px^{p-1}dx
\end{split}
\end{equation*}

The probability density function of swap rate can be derived from the 2nd order derivative of IRR-settled swaption price:
\begin{equation*}
\begin{split}
    f(s) &= \dfrac{1}{D(0,T)\cdot IRR(s)}\frac{\partial^2 V^{IRR}(s)}{\partial s^2}
\end{split}
\end{equation*}

Then work out static replication formula
\begin{equation*}
\begin{split}
    V_0 &= D(0,T) \int_K^\infty g(K,x)\dfrac{1}{D(0,T)\cdot IRR(x^p)}\frac{\partial^2 V^{IRR}(x^p)}{\partial (x^p)^2}\cdot px^{p-1}dx
\end{split}
\end{equation*}
Let $h(x) = \dfrac{g(x)}{IRR(x^p)}, h(K) = 0$ as $g(K) = 0$, hence
\begin{equation*}
\begin{split}
    V_0 &= \int_K^\infty h(x)\frac{\partial^2 V^{pay}_0}{\partial x^2}dx \\
    &= -h(K)\frac{\partial V^{pay}_0(K)}{\partial K} + h'(K)V^{pay}_0(K) + \int_K^\infty h''(x)V^{pay}_0(x)dx \\
    &= h'(K)V^{pay}_0(K) + \int_K^\infty h''(x)V^{pay}_0(x)dx 
\end{split}
\end{equation*}
Here
\begin{equation*}
\begin{split}
    h(K) &= \dfrac{g(K)}{IRR(K^p)} \\
    h'(K) &= \dfrac{IRR(K^p)g'(K)-pK^{p-1}IRR'(K^p)g(K)}{IRR(K^p)^2} \\
    h''(K) &= \dfrac{IRR(K^p)g''(K)-p(p-1)K^{p-2}IRR'(K^p)g(K)-p^2K^{2p-2}IRR''(K^p)g(K)-2pK^{p-1}IRR'(K^p)g'(K)}{IRR(K^p)^2} \\
    &\;\;\;\;\;\;\;\;\;\; + \dfrac{2p^2K^{2p-2}IRR'(K^p)^2g(K)}{IRR(K^p)^3}
\end{split}
\end{equation*} -->

<!-- We first work out the probability density function using Leibniz's rule:
\begin{equation*}
\begin{split}
    I(k) &= \int_{u(k)}^{v(k)} g(k, s)ds \\
    \frac{\partial I(k)}{\partial x} &= g(k, v(k))\dfrac{dv}{dk} - g(k, u(k))\dfrac{du}{dk} + \int_{u(k)}^{v(k)} \frac{\partial g(k, s)}{\partial k}ds
\end{split}
\end{equation*}
Apply to $V^{pay}_0$ we get
\begin{equation*}
\begin{split}
    \frac{\partial V^{pay}_0}{\partial K} &= D(0,T)\{IRR(\infty^p)g(K, \infty)f(\infty)\dfrac{d\infty}{dK} - IRR(K^p)g(K,K)f(K)\dfrac{dK}{dK} - \int_K^\infty IRR(x^p)f(x)ds\} \\
    &= -D(0,T) \int_K^\infty IRR(x^p)f(x)ds \\
    \frac{\partial^2 V^{pay}_0}{\partial K^2} &= -D(0,T)\{IRR(\infty^p)f(\infty)\dfrac{d\infty}{dK} - IRR(K^p)f(K)\dfrac{dK}{dK} + \int_K^\infty \frac{\partial [IRR(x^p)f(x)]}{\partial K}ds\} \\
    &= D(0,T)\cdot IRR(K^p)\cdot f(K) \\
    => f(K) &= \dfrac{1}{D(0,T)\cdot IRR(K^p)}\frac{\partial^2 V^{pay}_0}{\partial K^2}
\end{split}
\end{equation*} -->
